In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import json

from datetime import datetime, timedelta

In [20]:
# Read the Parquet file
df = pd.read_parquet('Binance_BTCUSDT_20241022_121148_5.parquet')

# Print the DataFrame
df

,ts,bids,asks
0,2024-10-22 12:11:43.969,"[[1000.000, 141.472], [5000.000, 6.691], [3300...","[[67328.500, 8.920], [67329.800, 0.017], [6733..."
1,2024-10-22 12:11:44.225,"[[1000.000, 141.322], [5000.000, 6.711], [2000...","[[67328.500, 8.904], [67329.800, 0.015], [6732..."
2,2024-10-22 12:11:44.477,"[[1000.000, 141.322], [5000.000, 6.691], [5386...","[[67328.500, 9.022], [67329.800, 0.015], [6733..."
3,2024-10-22 12:11:44.734,"[[1000.000, 141.322], [5000.000, 6.711], [2000...","[[67328.500, 8.873], [67329.900, 0.557], [6733..."
4,2024-10-22 12:11:44.984,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[67328.500, 8.870], [67331.900, 1.029], [6733..."
5,2024-10-22 12:11:45.235,"[[1000.000, 141.322], [5000.000, 6.691], [5386...","[[67331.600, 0.018], [67331.900, 1.608], [6733..."
6,2024-10-22 12:11:45.486,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[67328.500, 8.865], [67328.600, 0.004], [6732..."
7,2024-10-22 12:11:45.736,"[[1000.000, 141.322], [5000.000, 6.691], [3300...","[[67328.500, 8.857], [67329.900, 0.557], [6733..."
8,2024-10-22 12:11:46.001,"[[1000.000, 141.322], [5000.000, 6.671], [3300...","[[67328.500, 8.898], [67329.900, 0.882], [6733..."
9,2024-10-22 12:11:46.272,"[[1000.000, 141.322], [5000.000, 6.711], [3300...","[[67328.500, 8.898], [67328.800, 0.015], [6732..."


In [21]:
# Initialize the new dataframe 'dfs' with the 'ts' column
dfs = pd.DataFrame()
dfs['ts'] = df['ts']

# Extract the first and last bid prices (first element of each list in 'bids')
dfs['first_bid'] = df['bids'].apply(lambda x: x[0][0] if len(x) > 0 else None)
dfs['last_bid'] = df['bids'].apply(lambda x: x[-1][0] if len(x) > 0 else None)

# Extract the first and last ask prices (first element of each list in 'asks')
dfs['first_ask'] = df['asks'].apply(lambda x: x[0][0] if len(x) > 0 else None)
dfs['last_ask'] = df['asks'].apply(lambda x: x[-1][0] if len(x) > 0 else None)

# Get the max of the first element in the 'bids' nested list
dfs['max_bid'] = df['bids'].apply(lambda x: max([i[0] for i in x]) if len(x) > 0 else None)

# Get the min of the first element in the 'asks' nested list
dfs['min_ask'] = df['asks'].apply(lambda x: min([i[0] for i in x]) if len(x) > 0 else None)

# Checking the value
dfs['max_last_bid'] = dfs['last_bid'] == dfs['max_bid']
dfs['min_first_ask'] = dfs['first_ask'] == dfs['min_ask']

# Show the result
dfs

,ts,first_bid,last_bid,first_ask,last_ask,max_bid,min_ask,max_last_bid,min_first_ask
0,2024-10-22 12:11:43.969,1000.000,67328.400,67328.500,68742.400,67328.400,67328.500,True,True
1,2024-10-22 12:11:44.225,1000.000,67370.000,67328.500,68910.700,67370.000,67328.500,True,True
2,2024-10-22 12:11:44.477,1000.000,67328.400,67328.500,68520.200,67328.400,67328.500,True,True
3,2024-10-22 12:11:44.734,1000.000,67328.400,67328.500,154041.700,67328.400,67328.500,True,True
4,2024-10-22 12:11:44.984,1000.000,67328.400,67328.500,68001.800,67328.400,67328.500,True,True
5,2024-10-22 12:11:45.235,1000.000,67328.400,67331.600,68064.800,67328.400,67331.600,True,True
6,2024-10-22 12:11:45.486,1000.000,67370.000,67328.500,71704.900,67370.000,67328.500,True,True
7,2024-10-22 12:11:45.736,1000.000,67328.400,67328.500,156225.900,67328.400,67328.500,True,True
8,2024-10-22 12:11:46.001,1000.000,67327.000,67328.500,68264.400,67327.000,67328.500,True,True
9,2024-10-22 12:11:46.272,1000.000,67328.400,67328.500,69007.100,67328.400,67328.500,True,True


In [25]:
# Filter rows where 'last_bid' is None or 'first_ask' is None
false_dfs = dfs[(dfs['last_bid'].isna()) | (dfs['first_ask'].isna())]

# Show the result
false_dfs

,ts,first_bid,last_bid,first_ask,last_ask,max_bid,min_ask,max_last_bid,min_first_ask


In [26]:
dfs['spread'] = dfs['min_ask'] - dfs['max_bid']
dfs

,ts,first_bid,last_bid,first_ask,last_ask,max_bid,min_ask,max_last_bid,min_first_ask,spread
0,2024-10-22 12:11:43.969,1000.000,67328.400,67328.500,68742.400,67328.400,67328.500,True,True,0.100
1,2024-10-22 12:11:44.225,1000.000,67370.000,67328.500,68910.700,67370.000,67328.500,True,True,-41.500
2,2024-10-22 12:11:44.477,1000.000,67328.400,67328.500,68520.200,67328.400,67328.500,True,True,0.100
3,2024-10-22 12:11:44.734,1000.000,67328.400,67328.500,154041.700,67328.400,67328.500,True,True,0.100
4,2024-10-22 12:11:44.984,1000.000,67328.400,67328.500,68001.800,67328.400,67328.500,True,True,0.100
5,2024-10-22 12:11:45.235,1000.000,67328.400,67331.600,68064.800,67328.400,67331.600,True,True,3.200
6,2024-10-22 12:11:45.486,1000.000,67370.000,67328.500,71704.900,67370.000,67328.500,True,True,-41.500
7,2024-10-22 12:11:45.736,1000.000,67328.400,67328.500,156225.900,67328.400,67328.500,True,True,0.100
8,2024-10-22 12:11:46.001,1000.000,67327.000,67328.500,68264.400,67327.000,67328.500,True,True,1.500
9,2024-10-22 12:11:46.272,1000.000,67328.400,67328.500,69007.100,67328.400,67328.500,True,True,0.100
